In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/팀우이홍황')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
% pwd

'/content/drive/MyDrive/팀우이홍황'

## Import

In [ ]:
import torch,gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import numpy as np
import random
import os
import math

from glob import glob
import pandas as pd
import cv2
from PIL import Image
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import torchvision.models as models
from torchvision import transforms

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
torch.cuda.is_available()

True

## Hyperparameters Setting

In [ ]:
CFG = {
    'IMG_SIZE':300,
    'EPOCHS':30,
    'LEARNING_RATE':0,
    'BATCH_SIZE':8,
    'SEED':41
}

## Fix RandomSeed

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [ ]:
def get_train_data(data_dir):
    img_path_list = []
    label_list = []
    for case_name in sorted(os.listdir(data_dir)):
        current_path = os.path.join(data_dir, case_name)
        if os.path.isdir(current_path):
            # get image path
            img_path_list.extend(sorted(glob(os.path.join(current_path, 'image', '*g'))))
            
            # get label
            label_df = pd.read_csv(current_path+'/label.csv')
            label_list.extend(label_df['leaf_weight'])
                
    return img_path_list, label_list

def get_test_data(data_dir):
    # get image path
    img_path_list = glob(os.path.join(data_dir, 'image', '*.jpg'))
    img_path_list.extend(glob(os.path.join(data_dir, 'image', '*.png')))
    img_path_list.sort(key=lambda x:str(x.split('/')[-1].split('.')[0]))
    return img_path_list

In [ ]:
all_img, all_lab = get_train_data('./dataset/train')
test_img_path = get_test_data('./dataset/test')

In [ ]:
D_Outlier = ['CASE36_01','CASE39_01','CASE37_01','CASE26_01','CASE11_01','CASE49_03',
             'CASE22_07','CASE61_02','CASE22_09','CASE22_10','CASE56_11','CASE57_10',
             'CASE18_09','CASE22_11','CASE60_34','CASE60_33','CASE60_32','CASE73_34',
             'CASE60_31','CASE60_30','CASE60_29','CASE60_28','CASE60_27','CASE26_45',
             'CASE26_46','CASE60_26','CASE70_33','CASE60_25','CASE26_44','CASE26_47',
             'CASE26_41','CASE26_43','CASE26_42','CASE26_40','CASE26_35','CASE73_31',
             'CASE23_02','CASE51_21','CASE73_14','CASE73_30','CASE71_30','CASE69_19',
             'CASE73_01','CASE73_10','CASE71_31','CASE45_17','CASE70_24','CASE70_23',
             'CASE23_03','CASE70_22','CASE70_21','CASE23_04','CASE23_08','CASE23_06',
             'CASE23_07','CASE23_05','CASE23_09','CASE17_02','CASE09_08','CASE52_01',
             'CASE57_19','CASE69_20','CASE02_10','CASE70_20','CASE70_18','CASE70_03',
             'CASE04_09','CASE70_17','CASE70_11','CASE70_12','CASE70_13'] #  이상치


In [ ]:
D_Outlier.sort()

In [ ]:
D_Outlier

In [ ]:
df = pd.DataFrame(all_img)
df

,0
0,./dataset/train/CASE01/image/CASE01_01.png
1,./dataset/train/CASE01/image/CASE01_02.png
2,./dataset/train/CASE01/image/CASE01_03.png
3,./dataset/train/CASE01/image/CASE01_04.png
4,./dataset/train/CASE01/image/CASE01_05.png
...,...
1587,./dataset/train/CASE74/image/CASE74_25.png
1588,./dataset/train/CASE74/image/CASE74_26.png
1589,./dataset/train/CASE74/image/CASE74_27.png
1590,./dataset/train/CASE75/image/CASE75_01.png


In [ ]:
df['label'] = all_lab
df

,0,label
0,./dataset/train/CASE01/image/CASE01_01.png,49.193
1,./dataset/train/CASE01/image/CASE01_02.png,59.764
2,./dataset/train/CASE01/image/CASE01_03.png,72.209
3,./dataset/train/CASE01/image/CASE01_04.png,85.737
4,./dataset/train/CASE01/image/CASE01_05.png,102.537
...,...,...
1587,./dataset/train/CASE74/image/CASE74_25.png,211.497
1588,./dataset/train/CASE74/image/CASE74_26.png,214.116
1589,./dataset/train/CASE74/image/CASE74_27.png,214.293
1590,./dataset/train/CASE75/image/CASE75_01.png,210.872


In [ ]:
df = df.rename(columns = {0:'img_path'})
df

,img_path,label
0,./dataset/train/CASE01/image/CASE01_01.png,49.193
1,./dataset/train/CASE01/image/CASE01_02.png,59.764
2,./dataset/train/CASE01/image/CASE01_03.png,72.209
3,./dataset/train/CASE01/image/CASE01_04.png,85.737
4,./dataset/train/CASE01/image/CASE01_05.png,102.537
...,...,...
1587,./dataset/train/CASE74/image/CASE74_25.png,211.497
1588,./dataset/train/CASE74/image/CASE74_26.png,214.116
1589,./dataset/train/CASE74/image/CASE74_27.png,214.293
1590,./dataset/train/CASE75/image/CASE75_01.png,210.872


In [ ]:
len(D_Outlier)

71

In [ ]:
for i in df['img_path']:
    x = i[-13:-4]
    if x in D_Outlier:
        indexNames = df[ df['img_path'] == i ].index
        # Delete these row indexes from dataFrame
        df.drop(indexNames , inplace=True)

In [ ]:
df

,img_path,label
0,./dataset/train/CASE01/image/CASE01_01.png,49.193
1,./dataset/train/CASE01/image/CASE01_02.png,59.764
2,./dataset/train/CASE01/image/CASE01_03.png,72.209
3,./dataset/train/CASE01/image/CASE01_04.png,85.737
4,./dataset/train/CASE01/image/CASE01_05.png,102.537
...,...,...
1587,./dataset/train/CASE74/image/CASE74_25.png,211.497
1588,./dataset/train/CASE74/image/CASE74_26.png,214.116
1589,./dataset/train/CASE74/image/CASE74_27.png,214.293
1590,./dataset/train/CASE75/image/CASE75_01.png,210.872


In [ ]:
all_img_path = list(df['img_path'])
all_label = list(df['label'])

## Train / Validation Split

In [ ]:
# Train : Validation = 0.8 : 0.2 Split
train_len = int(len(all_img_path)*0.8)

train_img_path = all_img_path[:train_len]
train_label = all_label[:train_len]

vali_img_path = all_img_path[train_len:]
vali_label = all_label[train_len:]

## CustomDataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, train_mode=True, transforms=None):
        self.transforms = transforms
        self.train_mode = train_mode
        self.img_path_list = img_path_list
        self.label_list = label_list

    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        # Get image data
        image = Image.open(img_path).convert('RGB')
        if self.transforms is not None:
            image = self.transforms(image)

        if self.train_mode:
            label = self.label_list[index]
            return image, label
        else:
            return image
    
    def __len__(self):
        return len(self.img_path_list)

In [ ]:
train_transformer = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize((CFG['IMG_SIZE'],CFG['IMG_SIZE'])),
                    transforms.RandomHorizontalFlip(),
                    transforms.RandomVerticalFlip(),
                    transforms.Normalize(mean=(0.8178337, 0.77571064, 0.7621395), std=(0.8030876, 0.76963955, 0.7339688))
                    ])

test_transformer = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize((CFG['IMG_SIZE'],CFG['IMG_SIZE'])),
                    transforms.Normalize(mean=(0.8178337, 0.77571064, 0.7621395), std=(0.8030876, 0.76963955, 0.7339688))
                    ]) 

In [ ]:
train_ds = CustomDataset(train_img_path, train_label, train_mode=True, transforms=train_transformer)
vali_ds = CustomDataset(vali_img_path, vali_label, train_mode=True, transforms=train_transformer)
train_loader = DataLoader(train_ds, batch_size=CFG['BATCH_SIZE'], shuffle=True)
vali_loader = DataLoader(vali_ds, batch_size=CFG['BATCH_SIZE'], shuffle=True)

## Train

In [ ]:
def train(model, optimizer, train_loader, vali_loader, scheduler, device):
    model.to(device)

    # Loss Function
    criterion = nn.L1Loss().to(device)
    best_mae = 9999
    
    for epoch in range(1,CFG["EPOCHS"]+1):
        model.train()
        train_loss = []
        for img, label in tqdm(iter(train_loader)):
            img, label = img.float().to(device), label.float().to(device)
            
            optimizer.zero_grad()

            # Data -> Model -> Output
            logit = model(img)
            # Calc loss
            loss = criterion(logit.squeeze(1), label)

            # backpropagation
            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())
            
        # Evaluation Validation set
        vali_mae = validation(model, vali_loader, criterion, device)
        scheduler.step(vali_mae)
        
        print(f'Epoch [{epoch}] Train MAE : [{np.mean(train_loss):.5f}] Validation MAE : [{vali_mae:.5f}]\n')
        
        # Model Saved
        if best_mae > vali_mae:
            best_mae = vali_mae
            torch.save(model.state_dict(), './saved/best_effnet.pth')
            print('Model Saved.')

In [ ]:
def validation(model, vali_loader, criterion, device):
    model.eval() # Evaluation
    vali_loss = []
    with torch.no_grad():
        for img, label in tqdm(iter(vali_loader)):
            img, label = img.float().to(device), label.float().to(device)

            logit = model(img)
            loss = criterion(logit.squeeze(1), label)
            
            vali_loss.append(loss.item())

    vali_mae_loss = np.mean(vali_loss)
    return vali_mae_loss

## Efficientnet

In [ ]:
# Swish activation function
class Swish(nn.Module):
    def __init__(self):
        super().__init__()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return x * self.sigmoid(x)

# check
if __name__ == '__main__':
    x = torch.randn(3, 3, 224, 224)
    model = Swish()
    output = model(x)
    print('output size:', output.size())

output size: torch.Size([3, 3, 224, 224])


In [ ]:
# SE Block
class SEBlock(nn.Module):
    def __init__(self, in_channels, r=4):
        super().__init__()

        self.squeeze = nn.AdaptiveAvgPool2d((1,1))
        self.excitation = nn.Sequential(
            nn.Linear(in_channels, in_channels * r),
            Swish(),
            nn.Linear(in_channels * r, in_channels),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.squeeze(x)
        x = x.view(x.size(0), -1)
        x = self.excitation(x)
        x = x.view(x.size(0), x.size(1), 1, 1)
        return x

# check
if __name__ == '__main__':
    x = torch.randn(3, 56, 17, 17)
    model = SEBlock(x.size(1))
    output = model(x)
    print('output size:', output.size())

output size: torch.Size([3, 56, 1, 1])


In [ ]:
class MBConv(nn.Module):
    expand = 6
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, se_scale=4, p=0.5):
        super().__init__()
        # first MBConv is not using stochastic depth
        self.p = torch.tensor(p).float() if (in_channels == out_channels) else torch.tensor(1).float()

        self.residual = nn.Sequential(
            nn.Conv2d(in_channels, in_channels * MBConv.expand, 1, stride=stride, padding=0, bias=False),
            nn.BatchNorm2d(in_channels * MBConv.expand, momentum=0.99, eps=1e-3),
            Swish(),
            nn.Conv2d(in_channels * MBConv.expand, in_channels * MBConv.expand, kernel_size=kernel_size,
                      stride=1, padding=kernel_size//2, bias=False, groups=in_channels*MBConv.expand),
            nn.BatchNorm2d(in_channels * MBConv.expand, momentum=0.99, eps=1e-3),
            Swish()
        )

        self.se = SEBlock(in_channels * MBConv.expand, se_scale)

        self.project = nn.Sequential(
            nn.Conv2d(in_channels*MBConv.expand, out_channels, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(out_channels, momentum=0.99, eps=1e-3)
        )

        self.shortcut = (stride == 1) and (in_channels == out_channels)

    def forward(self, x):
        # stochastic depth
        if self.training:
            if not torch.bernoulli(self.p):
                return x

        x_shortcut = x
        x_residual = self.residual(x)
        x_se = self.se(x_residual)

        x = x_se * x_residual
        x = self.project(x)

        if self.shortcut:
            x= x_shortcut + x

        return x

# check
if __name__ == '__main__':
    x = torch.randn(3, 16, 24, 24)
    model = MBConv(x.size(1), x.size(1), 3, stride=1, p=1)
    model.train()
    output = model(x)
    x = (output == x)
    print('output size:', output.size(), 'Stochastic depth:', x[1,0,0,0])

output size: torch.Size([3, 16, 24, 24]) Stochastic depth: tensor(False)


In [ ]:
class SepConv(nn.Module):
    expand = 1
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, se_scale=4, p=0.5):
        super().__init__()
        # first SepConv is not using stochastic depth
        self.p = torch.tensor(p).float() if (in_channels == out_channels) else torch.tensor(1).float()

        self.residual = nn.Sequential(
            nn.Conv2d(in_channels * SepConv.expand, in_channels * SepConv.expand, kernel_size=kernel_size,
                      stride=1, padding=kernel_size//2, bias=False, groups=in_channels*SepConv.expand),
            nn.BatchNorm2d(in_channels * SepConv.expand, momentum=0.99, eps=1e-3),
            Swish()
        )

        self.se = SEBlock(in_channels * SepConv.expand, se_scale)

        self.project = nn.Sequential(
            nn.Conv2d(in_channels*SepConv.expand, out_channels, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(out_channels, momentum=0.99, eps=1e-3)
        )

        self.shortcut = (stride == 1) and (in_channels == out_channels)

    def forward(self, x):
        # stochastic depth
        if self.training:
            if not torch.bernoulli(self.p):
                return x

        x_shortcut = x
        x_residual = self.residual(x)
        x_se = self.se(x_residual)

        x = x_se * x_residual
        x = self.project(x)

        if self.shortcut:
            x= x_shortcut + x

        return x

# check
if __name__ == '__main__':
    x = torch.randn(3, 16, 24, 24)
    model = SepConv(x.size(1), x.size(1), 3, stride=1, p=1)
    model.train()
    output = model(x)
    # stochastic depth check
    x = (output == x)
    print('output size:', output.size(), 'Stochastic depth:', x[1,0,0,0])

output size: torch.Size([3, 16, 24, 24]) Stochastic depth: tensor(False)


In [ ]:
class EfficientNet(nn.Module):
    def __init__(self, num_classes=1, width_coef=1., depth_coef=1., scale=1., dropout=0.2, se_scale=4, stochastic_depth=False, p=0.5):
        super().__init__()
        channels = [32, 16, 24, 40, 80, 112, 192, 320, 1280]
        repeats = [1, 2, 2, 3, 3, 4, 1]
        strides = [1, 2, 2, 2, 1, 2, 1]
        kernel_size = [3, 3, 5, 3, 5, 5, 3]
        depth = depth_coef
        width = width_coef

        channels = [int(x*width) for x in channels]
        repeats = [int(x*depth) for x in repeats]

        # stochastic depth
        if stochastic_depth:
            self.p = p
            self.step = (1 - 0.5) / (sum(repeats) - 1)
        else:
            self.p = 1
            self.step = 0


        # efficient net
        self.upsample = nn.Upsample(scale_factor=scale, mode='bilinear', align_corners=False)

        self.stage1 = nn.Sequential(
            nn.Conv2d(3, channels[0],3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(channels[0], momentum=0.99, eps=1e-3)
        )

        self.stage2 = self._make_Block(SepConv, repeats[0], channels[0], channels[1], kernel_size[0], strides[0], se_scale)

        self.stage3 = self._make_Block(MBConv, repeats[1], channels[1], channels[2], kernel_size[1], strides[1], se_scale)

        self.stage4 = self._make_Block(MBConv, repeats[2], channels[2], channels[3], kernel_size[2], strides[2], se_scale)

        self.stage5 = self._make_Block(MBConv, repeats[3], channels[3], channels[4], kernel_size[3], strides[3], se_scale)

        self.stage6 = self._make_Block(MBConv, repeats[4], channels[4], channels[5], kernel_size[4], strides[4], se_scale)

        self.stage7 = self._make_Block(MBConv, repeats[5], channels[5], channels[6], kernel_size[5], strides[5], se_scale)

        self.stage8 = self._make_Block(MBConv, repeats[6], channels[6], channels[7], kernel_size[6], strides[6], se_scale)

        self.stage9 = nn.Sequential(
            nn.Conv2d(channels[7], channels[8], 1, stride=1, bias=False),
            nn.BatchNorm2d(channels[8], momentum=0.99, eps=1e-3),
            Swish()
        ) 

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.dropout = nn.Dropout(p=dropout)
        self.linear = nn.Linear(channels[8], num_classes)

    def forward(self, x):
        x = self.upsample(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)
        x = self.stage5(x)
        x = self.stage6(x)
        x = self.stage7(x)
        x = self.stage8(x)
        x = self.stage9(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.linear(x)
        return x


    def _make_Block(self, block, repeats, in_channels, out_channels, kernel_size, stride, se_scale):
        strides = [stride] + [1] * (repeats - 1)
        layers = []
        for stride in strides:
            layers.append(block(in_channels, out_channels, kernel_size, stride, se_scale, self.p))
            in_channels = out_channels
            self.p -= self.step

        return nn.Sequential(*layers)


def efficientnet_b0(num_classes=1):
    return EfficientNet(num_classes=num_classes, width_coef=1.0, depth_coef=1.0, scale=1.0,dropout=0.2, se_scale=4)

def efficientnet_b1(num_classes=1):
    return EfficientNet(num_classes=num_classes, width_coef=1.0, depth_coef=1.1, scale=240/224, dropout=0.2, se_scale=4)

def efficientnet_b2(num_classes=1):
    return EfficientNet(num_classes=num_classes, width_coef=1.1, depth_coef=1.2, scale=260/224., dropout=0.3, se_scale=4)

def efficientnet_b3(num_classes=1):
    return EfficientNet(num_classes=num_classes, width_coef=1.2, depth_coef=1.4, scale=300/224, dropout=0.3, se_scale=4)

def efficientnet_b4(num_classes=1):
    return EfficientNet(num_classes=num_classes, width_coef=1.4, depth_coef=1.8, scale=380/224, dropout=0.4, se_scale=4)

def efficientnet_b5(num_classes=1):
    return EfficientNet(num_classes=num_classes, width_coef=1.6, depth_coef=2.2, scale=456/224, dropout=0.4, se_scale=4)

def efficientnet_b6(num_classes=1):
    return EfficientNet(num_classes=num_classes, width_coef=1.8, depth_coef=2.6, scale=528/224, dropout=0.5, se_scale=4)

def efficientnet_b7(num_classes=1):
    return EfficientNet(num_classes=num_classes, width_coef=2.0, depth_coef=3.1, scale=600/224, dropout=0.5, se_scale=4)


# check
if __name__ == '__main__':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    x = torch.randn(3, 3, 224, 224).to(device)
    model = efficientnet_b3().to(device)
    output = model(x)
    print('output size:', output.size())

output size: torch.Size([3, 1])


In [ ]:
print model summary
from torchsummary import summary
model = efficientnet_b0().to(device)
summary(model, (3,224,224), device=device.type)

## Custom CosineAnnealingWarmRestarts

In [ ]:
import math
from torch.optim.lr_scheduler import _LRScheduler

class CosineAnnealingWarmUpRestarts(_LRScheduler):
    def __init__(self, optimizer, T_0, T_mult=1, eta_max=0.1, T_up=0, gamma=1., last_epoch=-1):
        if T_0 <= 0 or not isinstance(T_0, int):
            raise ValueError("Expected positive integer T_0, but got {}".format(T_0))
        if T_mult < 1 or not isinstance(T_mult, int):
            raise ValueError("Expected integer T_mult >= 1, but got {}".format(T_mult))
        if T_up < 0 or not isinstance(T_up, int):
            raise ValueError("Expected positive integer T_up, but got {}".format(T_up))
        self.T_0 = T_0
        self.T_mult = T_mult
        self.base_eta_max = eta_max
        self.eta_max = eta_max
        self.T_up = T_up
        self.T_i = T_0
        self.gamma = gamma
        self.cycle = 0
        self.T_cur = last_epoch
        super(CosineAnnealingWarmUpRestarts, self).__init__(optimizer, last_epoch)
    
    def get_lr(self):
        if self.T_cur == -1:
            return self.base_lrs
        elif self.T_cur < self.T_up:
            return [(self.eta_max - base_lr)*self.T_cur / self.T_up + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.eta_max - base_lr) * (1 + math.cos(math.pi * (self.T_cur-self.T_up) / (self.T_i - self.T_up))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.T_cur = self.T_cur + 1
            if self.T_cur >= self.T_i:
                self.cycle += 1
                self.T_cur = self.T_cur - self.T_i
                self.T_i = (self.T_i - self.T_up) * self.T_mult + self.T_up
        else:
            if epoch >= self.T_0:
                if self.T_mult == 1:
                    self.T_cur = epoch % self.T_0
                    self.cycle = epoch // self.T_0
                else:
                    n = int(math.log((epoch / self.T_0 * (self.T_mult - 1) + 1), self.T_mult))
                    self.cycle = n
                    self.T_cur = epoch - self.T_0 * (self.T_mult ** n - 1) / (self.T_mult - 1)
                    self.T_i = self.T_0 * self.T_mult ** (n)
            else:
                self.T_i = self.T_0
                self.T_cur = epoch
                
        self.eta_max = self.base_eta_max * (self.gamma**self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

## Run!!

In [ ]:
model = model

optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=2, T_mult=1, eta_max=0.1,  T_up=1, gamma=0.5)

train(model, optimizer, train_loader, vali_loader, scheduler, device)

  0%|          | 0/153 [00:00<?, ?it/s]

  0%|          | 0/39 [00:00<?, ?it/s]

Epoch [1] Train MAE : [80.76738] Validation MAE : [111.46408]

Model Saved.


  0%|          | 0/153 [00:00<?, ?it/s]

  0%|          | 0/39 [00:00<?, ?it/s]

...

  0%|          | 0/153 [00:00<?, ?it/s]

  0%|          | 0/39 [00:00<?, ?it/s]

Epoch [4] Train MAE : [81.05417] Validation MAE : [77.39017]



  0%|          | 0/153 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

# Inference

In [ ]:
def predict(model, test_loader, device):
    model.eval()
    model_pred = []
    with torch.no_grad():
        for img in tqdm(iter(test_loader)):
            img = img.float().to(device)

            pred_logit = model(img)
            pred_logit = pred_logit.squeeze(1).detach().cpu()

            model_pred.extend(pred_logit.tolist())
    return model_pred

In [ ]:
test_dataset = CustomDataset(test_img_path, None, train_mode=False, transforms=test_transformer)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

# Validation Score가 가장 뛰어난 모델을 불러옵니다.
checkpoint = torch.load('./saved/best_effnet.pth')
model = model
model.load_state_dict(checkpoint)

# Inference
preds = predict(model, test_loader, device)

## Submission

In [ ]:
submission = pd.read_csv('./dataset/sample_submission.csv')
submission['leaf_weight'] = preds
submission.to_csv('./submission/best_effnet.csv', index=False)